In [71]:
import pandas as pd
import numpy as np
import pdfplumber

import time
import os
import sys
from pathlib import Path

import re


import PyPDF2



In [37]:
def get_file_name(file_path):
    
    '''get files under certain path

    Args:
        file path,string

    Returns:
        A dictionary for stroing the file name and it's path

    Raise;
        ValueError,file path name

    '''
    file_dir ={}
    for dirpath, dirnames, files in os.walk(file_path, topdown=False): ## work down certain dir,has 3 returns
#         print(f'Found directory: {dirpath}')
        for file_name in files:
            file_dir[file_name] = os.path.join(dirpath,file_name)
#             print(os.path.join(dirpath,file_name))
            
    return file_dir

In [38]:
file_dir = get_file_name('./well_logs')

In [46]:
def get_survey_path():
    
    '''get file contain 'survey'

    Args:
        file path,string

    Returns:
        A dictionary for stroing the file name and it's path

    Raise;
     
    '''
    survey_path ={}
    pattern = r'.*[Ss]urvey' ##pattern
    for file,path in file_dir.items():
        find_file = re.search(pattern,file) ## find the file with such pattern
        if find_file:
            file_name = find_file.group(0) ##save the name for matching the pattern
            survey_path[file_name] = path ## save file name to dictionary
            
    return survey_path
            

In [50]:

def decrpyt_pdf(input_file,out_put_file,searching_item):
    '''decrpted file with qpdf from command line

    Args:
        input_file ,oup_put_file path and searching_item,string

    Returns:
        oup_put_file path,string

    Raise;
        file not found error
     
    '''
     
    PASSWORD = '123' ## set a random guess password
    try:
        with open(input_file, mode='rb') as f:  ## open the pdf file in rb mode 
            reader = PyPDF2.PdfFileReader(f) # create a pdffileread object
            if reader.isEncrypted: # check if the file is encryped or not
                try:
                    reader.decrypt(PASSWORD) ## give a random guess
                ## catch the wroing guess and decrypt the input file
                except NotImplementedError: 
                    command= f"qpdf  --decrypt {input_file} {out_put_file};" ## concate the command 
                    os.system(command) ## run the comman inside the cmd
                    ## open the decrypted file and perform function
                    with open(FILE_OUT_PATH, mode='rb') as fp:
                        reader = PyPDF2.PdfFileReader(fp) ## generate reader object
                        counters = read_page_numbers(reader,searching_item) ## perform function 
            else:
                ## if not encrypted, then perform searching function
                counters = read_page_numbers(reader,searching_item)
    ## catch file not found error
    except OSError as e:
        print(e)

In [47]:
survey_path = get_survey_path()

In [113]:
len(survey_path)

700

In [102]:
def get_survef_content(survey_path):

    '''get pdf content from the survey_path

    Args:
        survey_path, dict

    Returns:
        tables generator

    Raise;
        file not found error
    '''
    for file,path in survey_path.items():
        tables = [] ## table for saving one from the files
        pdf = pdfplumber.open(path)
        print('preprocess file of',path)
        for page in pdf.pages:
        #     print(page.extract_text())
            for pdf_table in page.extract_tables():
                table = []
                cells = []
                for row in pdf_table:
                    if not any(row):
                        # 如果一行全为空，则视为一条记录结束
                        if any(cells):
                            table.append(cells)
                            cells = []
                    elif all(row):
                        # 如果一行全不为空，则本条为新行，上一条结束
                        if any(cells):
                            table.append(cells)
                            cells = []
                        table.append(row)
                    else:
                        if len(cells) == 0:
                            cells = row
                        else:
                            for i in range(len(row)):
                                if row[i] is not None:
                                    cells[i] = row[i] if cells[i] is None else cells[i] + row[i]
                tables.append(table)
                for row in table:
                    print([re.sub('\s+', '', cell) if cell is not None else None for cell in row])
                print('---------- 分割线 ----------')


#                 pdf.close()

                yield tables


In [115]:
for tables in get_survef_content(survey_path):
    print(tables)
#     if len(tables) > 4:
#         break

preprocess file of ./well_logs\well_logs\01\trackingNo_203729\05535133 Gyro Survey001.pdf
preprocess file of ./well_logs\well_logs\01\trackingNo_203729\42-055-35133 Surveys.pdf
preprocess file of ./well_logs\well_logs\01\trackingNo_205798\Korn A 3H Surveys.pdf
---------- 分割线 ----------
[[]]
---------- 分割线 ----------
[[], []]
---------- 分割线 ----------
[[], [], []]
---------- 分割线 ----------
[[], [], [], []]
preprocess file of ./well_logs\well_logs\01\trackingNo_212488\MG LILLIE 1616 GU B 7H-Final Gyro Survey 0-4231.36.pdf
preprocess file of ./well_logs\well_logs\08\trackingNo_222775\Survey Report-Garrett-Snell Unit A 25-36 No. 1SH(Final)_.pdf
['*o8pany:SurgeEnergyLLC)oca3*o"or-inateRe9erence:,ellNo\'(SH$ro7ect:HowardCounty,TX(NAD27)%:DRe9erence:,ell-2#./\'//u01t(GL22##345672224(8att"#9))Site:Garrett-SnellntA2!-"#MDRe9erence:,ell-2#./\'//u01t(GL22##345672224(8att"#9))We33:No\'(SH0orthRe9erence:GrdWe33;ore:)HSurvey*a3cu3ationMetho-:*n+u+Cur$atureDesign:Gyro:*,D!3#4-(72//48T7(727"4Data;ase:

['*o8pany:$ro7ect:Site:We33:We33;ore:Design:', 'SurgeEnergyLLCHowardCounty,TX(NAD27)Garrett-SnellntA2!-"#No\'(SH)HGyro:*,D!3#4-(72//48T7(727"4', ')oca3*o"or-inateRe9erence:,ellNo\'(SH%:DRe9erence:,ell-2#./\'//u01t(GL22##345672224(8att"#9))MDRe9erence:,ell-2#./\'//u01t(GL22##345672224(8att"#9))0orthRe9erence:GrdSurvey*a3cu3ationMetho-:*n+u+Cur$atureData;ase:,ell8lanner(']
---------- 分割线 ----------
[[['*o8pany: Surge Energy LLC )oca3 *o"or-inate Re9erence: ,ell No\' (SH\n$ro7ect: Howard County, TX (NAD27) %:D Re9erence: ,ell - 2#./\'//u01t (GL2 2##34 5 672 224 (8att"#9))\nSite: Garrett-Snell \x1fn t A 2!-"# MD Re9erence: ,ell - 2#./\'//u01t (GL2 2##34 5 672 224 (8att"#9))\nWe33: No\' (SH 0orth Re9erence: Gr d\nWe33;ore: )H Survey *a3cu3ation Metho-: * n +u+ Cur$ature\nDesign: Gyro:*,D !3#4-(72//4 8T7 (727"4 Data;ase: ,ell8lanner('], ['$ro7ect Howard County, TX (NAD27)'], ['Map Syste8: \x1fS State 8lane (.27 (E;a<t 0olut on) Syste8 Datu8: *ean Sea Le$el\nGeo Datu8: NAD (.27 (NADC)N C)N\x1

['*o8pany:$ro7ect:Site:We33:We33;ore:Design:', 'SurgeEnergyLLCHowardCounty,TX(NAD27)Garrett-SnellntA2!-"#No\'(SH)HGyro:*,D!3#4-(72//48T7(727"4', ')oca3*o"or-inateRe9erence:,ellNo\'(SH%:DRe9erence:,ell-2#./\'//u01t(GL22##345672224(8att"#9))MDRe9erence:,ell-2#./\'//u01t(GL22##345672224(8att"#9))0orthRe9erence:GrdSurvey*a3cu3ationMetho-:*n+u+Cur$atureData;ase:,ell8lanner(']
---------- 分割线 ----------
[[['*o8pany: Surge Energy LLC )oca3 *o"or-inate Re9erence: ,ell No\' (SH\n$ro7ect: Howard County, TX (NAD27) %:D Re9erence: ,ell - 2#./\'//u01t (GL2 2##34 5 672 224 (8att"#9))\nSite: Garrett-Snell \x1fn t A 2!-"# MD Re9erence: ,ell - 2#./\'//u01t (GL2 2##34 5 672 224 (8att"#9))\nWe33: No\' (SH 0orth Re9erence: Gr d\nWe33;ore: )H Survey *a3cu3ation Metho-: * n +u+ Cur$ature\nDesign: Gyro:*,D !3#4-(72//4 8T7 (727"4 Data;ase: ,ell8lanner('], ['$ro7ect Howard County, TX (NAD27)'], ['Map Syste8: \x1fS State 8lane (.27 (E;a<t 0olut on) Syste8 Datu8: *ean Sea Le$el\nGeo Datu8: NAD (.27 (NADC)N C)N\x1

---------- 分割线 ----------
[[]]
['CLOSURECLOSUREMDINCAZIMTVDVS+N/-S+E/-WDLSDISTDIRftdegdegftftftft°/100ftftdeg']
['591.000.50208.12590.992.09-2.07-1.05Invalid2.32206.90710.000.43222.59709.992.89-2.86-1.600.113.27209.20805.000.81205.75804.983.77-3.72-2.130.444.29209.76899.000.40244.23898.984.52-4.46-2.710.595.22211.31994.000.75240.95993.974.99-4.91-3.560.366.06215.93']
['1089.000.37213.661088.975.56-5.47-4.270.486.94218.001183.002.55234.891182.937.05-6.92-6.152.359.26221.621278.004.71226.861277.7311.03-10.81-10.732.3315.23224.791373.006.15223.401372.3117.53-17.17-17.071.5624.21224.831468.006.66225.941466.7125.22-24.70-24.530.6134.81224.80']
['1563.006.79214.401561.0633.83-33.17-31.661.4345.85223.671657.006.69204.601654.4243.51-42.73-37.081.2356.58220.951752.007.13206.771748.7353.91-53.02-42.040.5467.67218.411847.006.56209.021843.0564.02-63.03-47.320.6678.82216.901942.006.71210.061937.4173.69-72.58-52.740.2189.72216.00']
['2036.006.65202.242030.7883.58-82.38-57.550.97100.49214.942131.007.

['CLOSURECLOSUREMDINCAZIMTVDVS+N/-S+E/-WDLSDISTDIRftdegdegftftftft°/100ftftdeg']
['8946.0090.87182.348328.691210.93-1203.92-342.323.621251.64195.879040.0092.95180.148325.561304.87-1297.83-344.353.221342.74194.869135.0092.95179.518320.661399.72-1392.71-344.060.661434.58193.889230.0089.09175.748318.961494.48-1487.58-340.125.681525.96192.88']
['9324.0091.91178.018318.141588.19-1581.42-334.993.851616.51191.969419.0090.91176.858315.811682.95-1676.29-330.721.611708.60191.169514.0089.23174.588315.701777.50-1771.01-323.622.971800.34190.369609.0092.02177.118314.661872.06-1865.74-316.743.961892.44189.649703.0089.70177.158313.261965.81-1959.61-312.042.471984.29189.05']
['9798.0087.11176.158315.902060.46-2054.40-306.492.922077.13188.499892.0093.32176.418315.542154.06-2148.15-300.406.612169.05187.969987.0095.14179.228308.532248.62-2242.81-296.783.522262.36187.5410082.0094.70183.128300.382343.25-2337.42-298.724.112356.43187.2810176.0093.33182.538293.802436.98-2431.08-303.331.592449.93187.11']
['1027

...

---------- 分割线 ----------
[[], [], [['CLOSURE CLOSURE\nMD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR\nft deg deg ft ft ft ft °/100ft ft deg'], ['0.00 0.00 0.00 0.00 0.00 0.00 0.00 Invalid 0.00 0.00\n175.00 0.59 334.48 175.00 0.87 0.81 -0.39 0.33 0.89 334.48\n236.00 0.27 299.39 236.00 1.31 1.16 -0.65 0.65 1.33 330.90\n326.00 0.70 302.97 325.99 2.03 1.56 -1.29 0.48 2.03 320.40\n417.00 1.32 276.80 416.98 3.29 1.99 -2.80 0.83 3.44 305.41'], ['508.00 2.04 277.45 507.94 5.18 2.33 -5.45 0.79 5.92 293.13\n599.00 3.44 274.78 598.83 8.18 2.76 -9.78 1.55 10.16 285.79\n693.00 5.18 262.69 692.56 12.24 2.46 -16.80 2.07 16.98 278.33\n787.00 6.79 241.60 786.06 15.30 -0.72 -25.90 2.88 25.91 268.40\n881.00 7.45 226.45 879.34 15.60 -7.56 -35.20 2.11 36.01 257.87'], ['974.00 7.47 216.52 971.56 13.36 -16.58 -43.17 1.38 46.24 248.99\n1069.00 8.19 211.71 1065.67 9.32 -27.30 -50.40 1.02 57.32 241.56\n1163.00 7.85 210.49 1158.75 4.61 -38.52 -57.18 0.40 68.95 236.03\n1258.00 7.59 208.93 1252.89 -0.27 -49.61 -63.5

---------- 分割线 ----------
[[['Railroad Commission of Texas', 'Oil and Gas DivisionP.O.Box 12967Capitol StationAustin TX, 78711', None, '', '', '', 'November 27, 2013Attn: Pam Johns', '', '', ''], [None, 'Enclosed, please find the original and one copy of the surveys performed on the referenced well byPremier Directional Drilling, LLC (Operator No. 676020) supervised by Ralph Clarke.  Other informationrequired by your office is as follows:', None, None, '', '', '', '', '', None], ['', 'JOB NUMBER', None, 'H13471', None, 'CUSTOMER', 'Rosetta Resources', None, None, ''], ['', 'WELL NAME', None, 'McGinty #2', None, 'COUNTY & STATE', None, 'Gonzales, Texas', None, ''], ['', 'API WELL NUMBER', None, '42-177-33169', None, 'PROPOSED DIRECTION', None, '107.72', None, ''], ['', 'TIE-IN DATAMEASURED \nDEPTH', 'VERTICAL \nDEPTH', 'INCLINATION', 'AZIMUTH', 'N-S', 'E-W', 'DATA SOURCE', None, ''], ['', 'FIRST\nSURVEYDATE10/20/2013', 'FIRST\nSURVEYDEPTH140 ft', 'INCLINATION.6°', 'AZIMUTH353.6°', '', '

['(cid:16)(cid:10)(cid:4)(cid:22)(cid:12)(cid:11)(cid:23)(cid:21)(cid:1)(cid:2)(cid:10)(cid:26)(cid:3)(cid:8)(cid:9)(cid:21)(cid:28)(cid:5)(cid:9)(cid:3)(cid:21)(cid:29)(cid:3)(cid:13)(cid:13)(cid:21)(cid:29)(cid:3)(cid:13)(cid:13)(cid:34)(cid:10)(cid:2)(cid:3)(cid:21)(cid:7)(cid:3)(cid:35)(cid:5)(cid:14)(cid:11)(cid:21)', '(cid:8)(cid:16)(cid:1)(cid:17)(cid:18)(cid:18)(cid:19)(cid:7)(cid:8)(cid:17)(cid:1)(cid:16)(cid:20)(cid:8)(cid:21)(cid:17)(cid:1)(cid:7)(cid:16)(cid:22)(cid:17)(cid:8)(cid:19)(cid:18)(cid:23)(cid:24)(cid:25)(cid:7)(cid:26)(cid:22)(cid:25)(cid:10)(cid:41)(cid:42)(cid:43)(cid:13)(cid:5)(cid:34)(cid:7)(cid:21)(cid:10)(cid:2)(cid:41)(cid:11)(cid:6)(cid:44)(cid:7)(cid:18)(cid:45)(cid:46)(cid:47)(cid:25)(cid:48)(cid:41)(cid:11)(cid:6)(cid:7)(cid:14)(cid:15)(cid:14)(cid:15)(cid:26)(cid:43)(cid:11)(cid:5)(cid:3)(cid:43)(cid:13)(cid:26)(cid:43)(cid:11)(cid:5)(cid:3)(cid:43)(cid:13)', '(cid:15)(cid:10)(cid:8)(cid:12)(cid:13)(cid:6)(cid:16)(cid:10)(cid:17)(cid:10)(cid:2)(cid:1

['(cid:16)(cid:10)(cid:4)(cid:22)(cid:12)(cid:11)(cid:23)(cid:21)(cid:1)(cid:2)(cid:10)(cid:26)(cid:3)(cid:8)(cid:9)(cid:21)(cid:28)(cid:5)(cid:9)(cid:3)(cid:21)(cid:29)(cid:3)(cid:13)(cid:13)(cid:21)(cid:29)(cid:3)(cid:13)(cid:13)(cid:34)(cid:10)(cid:2)(cid:3)(cid:21)(cid:7)(cid:3)(cid:35)(cid:5)(cid:14)(cid:11)(cid:21)', '(cid:8)(cid:16)(cid:1)(cid:17)(cid:18)(cid:18)(cid:19)(cid:7)(cid:8)(cid:17)(cid:1)(cid:16)(cid:20)(cid:8)(cid:21)(cid:17)(cid:1)(cid:7)(cid:16)(cid:22)(cid:17)(cid:8)(cid:19)(cid:18)(cid:23)(cid:24)(cid:25)(cid:7)(cid:26)(cid:22)(cid:25)(cid:10)(cid:41)(cid:42)(cid:43)(cid:13)(cid:5)(cid:34)(cid:7)(cid:21)(cid:10)(cid:2)(cid:41)(cid:11)(cid:6)(cid:44)(cid:7)(cid:18)(cid:45)(cid:46)(cid:47)(cid:25)(cid:48)(cid:41)(cid:11)(cid:6)(cid:7)(cid:14)(cid:15)(cid:14)(cid:15)(cid:26)(cid:43)(cid:11)(cid:5)(cid:3)(cid:43)(cid:13)(cid:26)(cid:43)(cid:11)(cid:5)(cid:3)(cid:43)(cid:13)', '(cid:15)(cid:10)(cid:8)(cid:12)(cid:13)(cid:6)(cid:16)(cid:10)(cid:17)(cid:10)(cid:2)(cid:1

KeyboardInterrupt: 

In [125]:
tables =[[], 
         [['CLOSURE CLOSURE\nMD INC AZIM TVD VS +N/-S +E/-W DLS DIST DIR\nft deg deg ft ft ft ft °/100ft ft deg'],
          ['591.00 0.50 208.12 590.99 2.09 -2.07 -1.05 Invalid 2.32 206.90\n710.00 0.43 222.59 709.99 2.89 -2.86 -1.60 0.11 3.27 209.20\n805.00 0.81 205.75 804.98 3.77 -3.72 -2.13 0.44 4.29 209.76\n899.00 0.40 244.23 898.98 4.52 -4.46 -2.71 0.59 5.22 211.31\n994.00 0.75 240.95 993.97 4.99 -4.91 -3.56 0.36 6.06 215.93'], ['1089.00 0.37 213.66 1088.97 5.56 -5.47 -4.27 0.48 6.94 218.00\n1183.00 2.55 234.89 1182.93 7.05 -6.92 -6.15 2.35 9.26 221.62\n1278.00 4.71 226.86 1277.73 11.03 -10.81 -10.73 2.33 15.23 224.79\n1373.00 6.15 223.40 1372.31 17.53 -17.17 -17.07 1.56 24.21 224.83\n1468.00 6.66 225.94 1466.71 25.22 -24.70 -24.53 0.61 34.81 224.80'], ['1563.00 6.79 214.40 1561.06 33.83 -33.17 -31.66 1.43 45.85 223.67\n1657.00 6.69 204.60 1654.42 43.51 -42.73 -37.08 1.23 56.58 220.95\n1752.00 7.13 206.77 1748.73 53.91 -53.02 -42.04 0.54 67.67 218.41\n1847.00 6.56 209.02 1843.05 64.02 -63.03 -47.32 0.66 78.82 216.90\n1942.00 6.71 210.06 1937.41 73.69 -72.58 -52.74 0.21 89.72 216.00'], ['2036.00 6.65 202.24 2030.78 83.58 -82.38 -57.55 0.97 100.49 214.94\n2131.00 7.07 193.30 2125.10 94.44 -93.16 -60.98 1.21 111.34 213.21\n2226.00 6.74 195.21 2219.41 105.57 -104.23 -63.79 0.42 122.20 211.47\n2321.00 7.59 203.93 2313.67 116.76 -115.35 -67.80 1.45 133.80 210.45\n2415.00 7.56 206.42 2406.85 128.08 -126.56 -73.07 0.35 146.13 210.00'], ['2510.00 7.37 206.03 2501.04 139.27 -137.63 -78.52 0.21 158.46 209.71\n2604.00 6.34 212.93 2594.37 149.17 -147.41 -83.99 1.40 169.66 209.67\n2699.00 6.47 211.17 2688.78 158.26 -156.39 -89.61 0.24 180.25 209.81\n2794.00 6.63 213.42 2783.16 167.54 -165.55 -95.40 0.32 191.07 209.95\n2888.00 6.74 216.60 2876.52 176.63 -174.50 -101.68 0.41 201.96 210.23'], ['2983.00 6.30 216.50 2970.91 185.43 -183.17 -108.10 0.46 212.69 210.55\n3078.00 6.40 216.93 3065.32 193.98 -191.59 -114.38 0.12 223.14 210.84\n3172.00 6.13 221.78 3158.76 202.05 -199.52 -120.87 0.63 233.28 211.21\n3267.00 6.30 220.91 3253.21 209.91 -207.24 -127.66 0.21 243.40 211.63\n3361.00 6.04 221.08 3346.66 217.67 -214.86 -134.29 0.28 253.37 212.01'], ['3456.00 5.26 219.29 3441.20 224.94 -222.00 -140.33 0.84 262.63 212.30\n3551.00 5.15 210.96 3535.81 232.07 -229.02 -145.28 0.80 271.22 212.39']]]

In [142]:
for table in tables:
    table_np =np.array(table)
    table_np_squ = table_np.squeeze()
#     print(table_np_squ)
    for tab in table_np_squ:
        if tab:
#             print(tab)
#             print(type(tab))
            for line in tab:
                split_line = tab.split(' ')
                print(split_line)
            

['CLOSURE', 'CLOSURE\nMD', 'INC', 'AZIM', 'TVD', 'VS', '+N/-S', '+E/-W', 'DLS', 'DIST', 'DIR\nft', 'deg', 'deg', 'ft', 'ft', 'ft', 'ft', '°/100ft', 'ft', 'deg']
['CLOSURE', 'CLOSURE\nMD', 'INC', 'AZIM', 'TVD', 'VS', '+N/-S', '+E/-W', 'DLS', 'DIST', 'DIR\nft', 'deg', 'deg', 'ft', 'ft', 'ft', 'ft', '°/100ft', 'ft', 'deg']
['CLOSURE', 'CLOSURE\nMD', 'INC', 'AZIM', 'TVD', 'VS', '+N/-S', '+E/-W', 'DLS', 'DIST', 'DIR\nft', 'deg', 'deg', 'ft', 'ft', 'ft', 'ft', '°/100ft', 'ft', 'deg']
['CLOSURE', 'CLOSURE\nMD', 'INC', 'AZIM', 'TVD', 'VS', '+N/-S', '+E/-W', 'DLS', 'DIST', 'DIR\nft', 'deg', 'deg', 'ft', 'ft', 'ft', 'ft', '°/100ft', 'ft', 'deg']
['CLOSURE', 'CLOSURE\nMD', 'INC', 'AZIM', 'TVD', 'VS', '+N/-S', '+E/-W', 'DLS', 'DIST', 'DIR\nft', 'deg', 'deg', 'ft', 'ft', 'ft', 'ft', '°/100ft', 'ft', 'deg']
['CLOSURE', 'CLOSURE\nMD', 'INC', 'AZIM', 'TVD', 'VS', '+N/-S', '+E/-W', 'DLS', 'DIST', 'DIR\nft', 'deg', 'deg', 'ft', 'ft', 'ft', 'ft', '°/100ft', 'ft', 'deg']
['CLOSURE', 'CLOSURE\nMD', 'INC', 

In [127]:
np_tab.shape

(1, 1, 0)